<a href="https://www.kaggle.com/code/rubanzasilva/fastai-neural-network-gradient-boosting?scriptVersionId=188472546" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Binary Classification of Insurance Cross 

The objective of this competition is to predict which customers respond positively to an automobile insurance offer.

A company that has been offering health insurance to a segment of customers wants to find out whether their health insurance customers are willing to also get/sign up for their motor vehicle insurance plan 

## Library Imports

In [1]:
%pip install catboost
%pip install optuna
%pip install optuna_distributed
%pip install openfe
%pip install seaborn
%pip install xgboost
%pip install lightgbm
%pip install fastkaggle
%pip install h2o
%pip install -Uqq fastbook
%pip install -q -U autogluon.tabular
%pip install autogluon
#ip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: distributed
    Found existing installation: distributed 2024.1.1
    Uninstalling distributed-2024.1.1:
      Successfully uninstalled distributed-2024.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rapids-dask-dependency 24.4.1a0 requires dask==2024.1.1, but you have dask 2024.5.2 which is incompatible.
rapids-dask-dependency 24.4.1a0 requires dask-expr==0.4.0, but you have dask-expr 1.1.2 which is incompatible.
rapids-dask-dependency 24.4.1a0 requires distributed==2024.1.1, but you have distributed 2024.5.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [3]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)


from pathlib import Path
import os


from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score



#transformers and pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv

import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
from optuna.samplers import TPESampler
import warnings


matplotlib.rc('image', cmap='Greys')

from fastkaggle import setup_comp



from openfe import OpenFE, transform
from autogluon.tabular import TabularDataset, TabularPredictor

#from IPython.display import FileLink

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
!ls /kaggle/input/

playground-series-s4e7


In [6]:
path = Path('/kaggle/input/playground-series-s4e7/')
path

Path('/kaggle/input/playground-series-s4e7')

In [7]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')
#original_df = pd.read_csv('/kaggle/input/academic-success-dataset/data.csv',delimiter=';')

In [20]:
train_df.shape,test_df.shape

((11504798, 11), (7669866, 10))

## Subset

In [9]:
train_subset = train_df.sample(n=1000000,replace=False)
test_subset = test_df.sample(n=500000,replace=False)

In [10]:
train_subset.shape,test_subset.shape

((1000000, 11), (500000, 10))

In [ ]:
#cont_names,cat_names = cont_cat_split(train_subset, dep_var='Response')
#splits = RandomSplitter(valid_pct=0.2)(range_of(train_subset))
#to = TabularPandas(train_subset, procs=[Categorify, FillMissing,Normalize],
#                  cat_names = cat_names,
#                  cont_names = cont_names,
#                  y_names='Response',
#                  y_block=CategoryBlock(),
#                  splits=splits)
#dls = to.dataloaders(bs=64)
#test_dl = dls.test_dl(test_subset)
#X_train, y_train = to.train.xs, to.train.ys.values.ravel()
#X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

# Full Dataset

Below i use the fastai cont_cat_split which returns the values of a particular dataframe as either categorical or continous values depending on the cardinality of the column values which is how many levels a particular column has.

This function takes an argument of max_card whose default is 20. If the number of unique values is above the max_card, then that variables is considered to be continuos and vice versa.

In [21]:
cont_names,cat_names = cont_cat_split(train_df, dep_var='Response')
len(cat_names),len(cont_names)

(5, 5)

In [22]:
cont_names

['Age', 'Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']

In [23]:
cat_names

['Gender',
 'Driving_License',
 'Previously_Insured',
 'Vehicle_Age',
 'Vehicle_Damage']

In [24]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))

In [25]:
to = TabularPandas(train_df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='Response',
                   y_block=CategoryBlock(),
                   splits=splits)

In [26]:
to.xs.iloc[:2]

,Gender,Driving_License,Previously_Insured,Vehicle_Age,Vehicle_Damage,Age,Region_Code,Annual_Premium,Policy_Sales_Channel,Vintage
id,,,,,,,,,,
246166,2,2,2,1,1,-0.692705,1.122448,-1.692401,0.806599,-0.498535
8126671,1,2,1,2,1,-1.092895,0.044648,-1.692401,0.732583,1.664421


In [27]:
dls = to.dataloaders(bs=64)
test_dl = dls.test_dl(test_df)

In [28]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

In [29]:
dls.show_batch()

,Gender,Driving_License,Previously_Insured,Vehicle_Age,Vehicle_Damage,Age,Region_Code,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Male,1,0,< 1 Year,Yes,23.000001,38.0,27953.000102,151.999998,167.000000,0
1,Male,1,0,1-2 Year,Yes,43.000000,3.0,2630.000866,25.999998,84.999999,0
2,Male,1,0,< 1 Year,Yes,24.000000,37.0,32658.999925,151.999998,94.000001,0
3,Male,1,0,1-2 Year,Yes,60.000001,46.0,2630.000866,124.000000,205.000001,1
4,Female,1,0,< 1 Year,Yes,20.999999,3.0,2630.000866,159.999999,153.000001,0
5,Female,1,1,< 1 Year,Yes,20.999999,30.0,2630.000866,159.999999,176.000000,0
6,Female,1,0,> 2 Years,Yes,54.000000,28.0,32055.999942,124.000000,68.000002,0
7,Male,1,0,1-2 Year,Yes,40.000000,28.0,35056.999888,124.000000,81.999997,0
8,Male,1,0,1-2 Year,Yes,45.000000,40.0,31535.999957,124.000000,278.999996,0
9,Female,1,1,1-2 Year,No,68.999999,28.0,27489.000108,29.000000,256.000000,0


# Neural Network

In [ ]:
learn = tabular_learner(dls, metrics=RocAucBinary())

In [ ]:
learn.lr_find(suggest_funcs=(slide,valley))

In [ ]:
%%time
learn.fit_one_cycle(8,0.005)


In [ ]:
dl = test_dl
nn_preds = learn.get_preds(dl=dl)
nn_preds_x = learn.get_preds()[0]
a_preds, _ = learn.get_preds(dl=dl)
nn_preds_y = a_preds.squeeze(1)
nn_preds_proba = (a_preds[:, 1])

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['Response'] = nn_preds_proba
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

In [ ]:
#argmax_c = target_preds.argmax(dim=1)

# Neural Network Ensemble

In [ ]:
def ensemble():
    learn = tabular_learner(dls, metrics=RocAucMulti())
    with learn.no_bar(),learn.no_logging(): learn.fit(5, 0.005)
    return learn.get_preds(test_dl=test_dl)[0]

In [ ]:
learns = [ensemble() for _ in range(5)]

In [ ]:
ens_preds = torch.stack(learns).mean(0)

# Random Forest

In [ ]:
%%time
rf = RandomForestClassifier(100, min_samples_leaf=3)
rf_model = rf.fit(X_train, y_train);

rf_preds = tensor(rf_model.predict(test_dl.xs))

rf_preds_x = tensor(rf_model.predict(X_test))

#mse = mean_absolute_error(y_test, rf_preds_x)
#rmse = np.sqrt(mse)

#accuracy_score(y_test,rf_preds_x)
roc_auc_score(y_test,rf_preds_x)

In [ ]:
#RandomForestClassifier??

# Gradient Boosting

# XGBoost

In [ ]:
xgb_optuna_params = {
    'n_estimators': 929,
    'alpha': 2.287466581490129e-05,
    'subsample': 0.8766675651018592,
    'colsample_bytree': 0.288332829334817,
    'max_depth': 8,
    'min_child_weight': 6,
    'learning_rate': 0.024083411832750343,
    'gamma': 0.001816649055813574
}

In [ ]:
print(xgb.get_config())

In [36]:
xgb_params = {
    'colsample_bylevel': 0.21416299403295808,
    'colsample_bynode': 0.6208356380953189,
    'colsample_bytree': 0.19219680400212635,
    'gamma': 0.6051664515971382,
    'max_bin': 682,
    'max_delta_step': 5.264818337431145,
    'max_depth': 68,
    'min_child_weight': 5.23408291542125,
    'n_estimators': 10000,
    'n_jobs': -1,
    'objective': "binary:logistic",
    'reg_alpha': 0.46516016901463414,
    'reg_lambda': 0.8410553418334902,
    'subsample': 0.802533192662325,
    'verbosity': 0,
    'eval_metric': "auc",
    #'early_stopping_rounds': 200,
    'tree_method': 'gpu_hist'  # Use GPU acceleration
}

In [37]:
%%time
xgb_model = xgb.XGBClassifier(**xgb_params)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))
xgb_preds_proba = tensor(xgb_model.predict_proba(test_dl.xs))[:, 1]

xgb_preds_x = tensor(xgb_model.predict(X_test))
xgb_preds_x_proba = tensor(xgb_model.predict_proba(X_test))[:, 1]

xgb_score = roc_auc_score(y_test,xgb_preds_x_proba)
xgb_score

0.556070909689183

In [39]:
xgb_score = roc_auc_score(y_test,xgb_preds_x_proba)
xgb_score

0.8783475718669381

In [41]:
!ls

requirements.txt


In [42]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['Response'] = xgb_preds_proba
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

,id,Response
0,11504798,0.001680
1,11504799,0.494707
2,11504800,0.240503
3,11504801,0.000181
4,11504802,0.093105
...,...,...
7669861,19174659,0.241956
7669862,19174660,0.000297
7669863,19174661,0.000345
7669864,19174662,0.595853


### Cross validation implementation

In [ ]:
scores = cross_val_score(xgb_model,X_train, y_train,cv=5,scoring='roc_auc')
scores

In [ ]:
scores.mean()

In [ ]:
xgb_score

In [ ]:
xgb_model = xgb.XGBClassifier(alpha=0.2271025051979102,
                    gamma=0.07819541270693704,
                    eta=0.294535700691525,
                    max_depth=15,
                    max_bin=174857,
                    min_child_weight=1,
                    max_child_weight=10,
                    n_estimators=9500,
                    subsample=0.8362024536878123,
                    tree_method="hist",
                    device="cuda",
                    use_label_encoder=False,
                    enable_categorical=True,
                    eval_metric='auc',
                    objective='binary:logistic',
                    colsample_bytree=0.42784607351667425)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))

xgb_preds_x = tensor(xgb_model.predict(X_test))

#accuracy_score(y_test,xgb_preds_x)
xgb_score = roc_auc_score(y_test,xgb_preds_x)
xgb_score

# LightGBM

In [ ]:
lgb_optuna_params =  {
    'num_leaves': 485, 
    'learning_rate': 0.016388605840878773, 
    'n_estimators': 1064, 
    'subsample_for_bin': 106545, 
    'min_child_samples': 382, 
    'reg_alpha': 4.355786683676367e-05, 
    'reg_lambda': 0.12174022484031638, 
    'colsample_bytree': 0.44894475300776, 
    'subsample': 0.735046656897411, 
    'max_depth': 8
}

In [ ]:
lgb_model = lgb.LGBMClassifier(**lgb_optuna_params)
lgb_model = lgb_model.fit(X_train, y_train)

#test set preds
lgb_preds = tensor(lgb_model.predict(test_dl.xs))

#validation set preds
lgb_preds_x = tensor(lgb_model.predict(X_test))

lgb_score = roc_auc_score(y_test,lgb_preds_x)
lgb_score

# CatBoost

In [ ]:
cat_optuna_params = {
    
    'colsample_bylevel': 0.6383474716497279,
    'learning_rate': 0.09475494290429642,
    'random_strength': 0.07771221926568195,
    'max_bin': 490,
    'depth': 5,
    'l2_leaf_reg': 5,
    'boosting_type': 'Plain',
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.8429457747642737
    
}

In [ ]:
%%time
cat_model = CatBoostClassifier(**cat_optuna_params)
cat_model = cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

#test set preds
cat_preds = tensor(cat_model.predict(test_dl.xs))


#cat_preds_final = cat_preds.squeeze(1)

#validation set preds
cat_preds_x = tensor(cat_model.predict(X_test))

#cat_preds_x_final = cat_preds_x.squeeze(1)

#accuracy_score(y_test,cat_preds_x)

cat_score = roc_auc_score(y_test,cat_preds_x)
cat_score

# Model Predictions

In [ ]:
model_preds = {
    "random forests":roc_auc_score(y_test,rf_preds_x),
    "cat boost":cat_score,
    "lgbm":lgb_score,
    "xgboost":roc_auc_score(y_test,xgb_preds_x),   
}

#model_preds_a = model_preds.sort()
print(model_preds)

In [ ]:
cat_preds_x

# Model Ensemble

# Submission

In [ ]:
!ls

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit.Response = xgb_preds
submit.to_csv('submission.csv',index=False)
submit

In [ ]:
!ls

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['Response'] = xgb_preds
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub